# Nightlight Satellite Image Export
## Prerequisites
### Input

* CSV with 4 columns:
    * ID
    * year of survey
    * Latitude-Coordinates
    * Longitude-Coordinates

### Execution
#### main pipeline
1. create a pandas dataframe with all the information and delete all unnecessary entries (in our case it is all entries who are from 2011 or earlier)
2. creation of a range of year in string format (yyyy-mm-dd) instead a year in integer format
3. create an image with the median
4. get the areas of interest (AOI) with the Latitude and Longitude information in the dataframe
5. export the image with predefined AOIs and dimensions
    
Google Documentary says 'To get a block of pixels of predefined size (for example a 256x256 thumbnail image) that covers a region, specify dimensions and region'

In [ ]:
import ee
import pandas as pd
from typing import Tuple
import utils
import time
import wget
import zipfile

In [ ]:
#ee.Authenticate()
ee.Initialize()

In [ ]:
#CONSTANTS
#Path of CSV-File with columns ID,year,LATITUDE,LONGITUDE
csvpath = '../dataResearch/firstSample.csv'
#name all column names
#year
year = 'year'
#Latitude and Longitude Coordinates
LATNUM = 'LATNUM'
LONGNUM= 'LONGNUM'
#ID for Filenames
surveyid = 'ID'
#Export parameters
DRIVE_EXPORT_FOLDER = 'geotiff_raw'
# image export parameters
PROJECTION = 'EPSG:3857'  # see https://epsg.io/3857
#1100 pixel times 10m equals 11000mx11000m image
SCALE = 10                # export resolution: 10m/px
IMAGE_DIMENSION = 5500  #radius of the wanted image in m

In [ ]:
#create Dataframe with pandas
df = pd.read_csv(csvpath)
#create Dataframe where year is 2012 or higher
df = df[df.year >= 2012]
#iterate over the dataframe with the information about each cluster
for i in range(len(df)):
    #Google Earth Engine can only process 3000 Tasks at once, so we wait for the Earth Engine to finish the 2999 tasks and then start the next 2999
    if i%2999 == 0:
        input("Nächsten Bilder starten mit EINGABE/ENTER")
    print("starting Task Nr: " + str(i))
    #get the start- and enddates of the cluster out of the function in Utils-Script
    start_date,end_date = utils.surveyyear_to_range(df[year].iloc[i],satellitename='nl')
    # get the VIIRS image collection, we're using the "avg_rad" band
    dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filter(ee.Filter.date(start_date, end_date))
    nighttime = dataset.select('avg_rad').median()
    # get the coordinates based on the AOI
    lat = float(df[LATNUM].iloc[i].replace(',', '.'))
    lon = float(df[LONGNUM].iloc[i].replace(',', '.'))
    aoi = ee.Geometry.Point(lon,lat)
    coords = utils.point_to_box_coords(aoi=aoi,dimensionradius=IMAGE_DIMENSION)
    #get the name of the cluster for the filename
    name = df[surveyid].iloc[i]
    #create the task with the image, the region, where to save it, the filename,the scale and a description
    task = ee.batch.Export.image.toDrive(image=nighttime,region=coords,folder=DRIVE_EXPORT_FOLDER,fileNamePrefix=name,scale=SCALE,description=name)
    #start the task
    task.start()